LangChain Tool
```
Tool : llm에이전트가 수행할수 있는 외부기능이나 api를 말함(날씨정보나 웹검색,게산기)
이유 : gpt는 최신정보에 접근할수 없다.
구조 : 이름 설명 함수실행로직
툴 호출 방식 :  GPT-4 모델들은 OpenAI의 함수호출 기능을 통해 툴을 직접 호출할수 있다다
```
외부 API 호출 Tool 구현(날씨API, 뉴스API)

In [ ]:
%pip install langchain-core langchain-openai openai python-dotenv requests

In [4]:
# .env 로드
from dotenv import load_dotenv
import os
load_dotenv()

print(os.environ["OPEN_WEATHER_MAP"][:5])
print(os.environ["NEWSAPI_API_KEY"][:5])

83e83
7ca03


In [5]:
# 날씨정보 조회 함수
import re


def get_weather(city: str) -> str:
    import requests
    api_key = os.getenv("OPEN_WEATHER_MAP")
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q":city,
        "appid": api_key,
        "units": "metric",  # 섭씨온도
        "lang": "kr"  # 한국어
    }
    response = requests.get(url, params=params)
    data =  response.json()
    if data.get('cod') != 200:
        return "날씨 정보를 가져올 수 없습니다."
    temp = data['main']['temp']
    desc = data['weather'][0]['description']
    return f"{city}의 현재 기온은 {temp}도이며, 날씨는 {desc}입니다."

In [8]:
get_weather('seoul')

'seoul의 현재 기온은 22.76도이며, 날씨는 연무입니다.'

뉴스검색

In [9]:
from turtle import title

from numpy import source


def get_news(keyword:str) -> str:
    import requests
    api_key = os.getenv("NEWSAPI_API_KEY")
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": keyword,
        "apiKey": api_key,
        "language": "ko",
        "sortBy": "publishedAt",  #최신뉴스
        "pageSize": 1  # 최근 1건
    }
    response = requests.get(url, params=params)
    data = response.json()    
    articles = data.get('articles')
    if not articles:
        return "해당 키워드에 대한 뉴스가 없습니다."
    
    # 가장 첫번째 뉴스선택
    top_news = articles[0]
    title = top_news.get('title', '제목 없음')
    source = top_news.get('source', {}).get('name', '출처 없음')
    return f'{keyword}에 대한 최신 뉴스: "{title}" (출처: {source})'

In [12]:
print(get_news('취업'))

취업에 대한 최신 뉴스: "취업 시 나를 증명하고 싶다면..."스펙터"" (출처: Zdnet.co.kr)
